# Vecalign



1.   https://github.com/thompsonb/vecalign
2.   https://aclanthology.org/D19-1136.pdf







In [ ]:
old_stdout = 

In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install transformers==3.1.0
!pip install python-docx
!pip install stanza
!pip install awesome-align

import torch
import transformers
import itertools
import docx
import re
from bs4 import BeautifulSoup
import stanza 
import pandas as pd
import numpy as np
import json
import re 

tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')

     |████████████████████████████████| 884 kB 16.2 MB/s 
     |████████████████████████████████| 3.0 MB 37.9 MB/s 
     |████████████████████████████████| 1.2 MB 66.2 MB/s 
     |████████████████████████████████| 895 kB 59.0 MB/s 
     |████████████████████████████████| 5.6 MB 26.0 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=b7faa971a84f5249b77fddc18065d56054903b3b1a18fe1e057c7183420efca7
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx
     |████████████████████████████████| 432 kB 30.0 MB/s 
     |████████████████████████████████| 175 kB 57.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=d440c02e4ed29bdd89adc4452eedcf9c396f75df29087dcd94497b4cbbf09f29
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |███████

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
stanza.download('en')
stanza.download('fr')
stanza.download('la')

nlp_eng = stanza.Pipeline('en')
nlp_fra = stanza.Pipeline('fr')
nlp_la = stanza.Pipeline('la')

2022-03-14 13:11:45 INFO: Downloading default packages for language: en (English)...


2022-03-14 13:11:54 INFO: Finished downloading models and saved to /root/stanza_resources.


2022-03-14 13:11:54 INFO: Downloading default packages for language: fr (French)...


2022-03-14 13:12:13 INFO: Finished downloading models and saved to /root/stanza_resources.


2022-03-14 13:12:14 INFO: Downloading default packages for language: la (Latin)...


2022-03-14 13:12:20 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-03-14 13:12:20 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-03-14 13:12:20 INFO: Use device: cpu
2022-03-14 13:12:20 INFO: Loading: tokenize
2022-03-14 13:12:20 INFO: Loading: pos
2022-03-14 13:12:20 INFO: Loading: lemma
2022-03-14 13:12:20 INFO: Loading: depparse
2022-03-14 13:12:21 INFO: Loading: sentiment
2022-03-14 13:12:21 INFO: Loading: constituency
2022-03-14 13:12:22 INFO: Loading: ner
2022-03-14 13:12:22 INFO: Done loading processors!
2022-03-14 13:12:22 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos    

# English

In [ ]:
french_xml = open('/content/bodin.livrep.perseus-fre1.xml')
french_soup = BeautifulSoup(french_xml, 'lxml')

english_xml = open('/content/bodin.livrep.perseus-eng1.xml')
english_soup = BeautifulSoup(english_xml, 'lxml') 

In [ ]:
def chapFromTEI(soup):
  chap = {}

  for note in soup.find_all('note'):
    note.decompose() 
  # for head in soup.find_all('head'):
  #   head.decompose() Why no head?? 
  for i in range(len(soup.find_all('div', attrs={"type": "textpart", "subtype": "chapter"}))):
    for p in soup.find_all('div', attrs={"type": "textpart", "subtype": "chapter"})[i].find_all('p'):
      text = ''
      if f'Chapter {i+1}' in chap:
        chap[f'Chapter {i+1}'] = chap[f'Chapter {i+1}'] + p.get_text().replace('\n','').replace('                     ', ' ').replace('                  ', ' ').replace('     ', ' ').replace('    ', ' ')
      else:
        chap[f'Chapter {i+1}'] = p.get_text().replace('\n','').replace('                     ', ' ').replace('                  ', ' ').replace('     ', ' ').replace('    ', ' ')

  df = pd.DataFrame.from_dict(chap, orient='index')
  return df

In [ ]:
french_df = chapFromTEI(french_soup)
english_df = chapFromTEI(english_soup)

In [ ]:
fre_chap1 = french_df[0][0]
with open('dev.fr', 'w') as fDev:
  for line in sent_tokenize(fre_chap1, language='french'):
    fDev.write(line + '\n')


eng_chap1 = english_df[0][0]
with open('dev.en', 'w') as eDev:
  for line in sent_tokenize(eng_chap1, language='english'):
    eDev.write(line + '\n')

In [ ]:
## https://github.com/thompsonb/vecalign FIGURE OUT THIS!! 
!git clone https://github.com/thompsonb/vecalign

In [ ]:
!pip install cython

In [ ]:
!/content/vecalign/overlap.py -i /content/dev.fr -o overlaps.fr -n 10
!/content/vecalign/overlap.py -i /content/dev.en -o overlaps.en -n 10

In [ ]:
!pip install laserembeddings

In [ ]:
!python -m laserembeddings download-models
from laserembeddings import Laser

laser = Laser()

# embeddings = laser.embed_sentences(
#     ['let your neural network be polyglot',
#      'use multilingual embeddings!'],
#     lang='en')

In [ ]:
with open('/content/overlaps.fr', 'r') as frOver:
  fr_overlap = frOver.readlines() 

with open('/content/overlaps.en', 'r') as enOver:
  en_overlap = enOver.readlines()

In [ ]:
fr_embeddings = laser.embed_sentences(fr_overlap,lang='fr')

In [ ]:
en_embeddings = laser.embed_sentences(en_overlap, lang='en')

In [ ]:
with open('overlaps.fr.emb','wb') as fEmb:
  for line in fr_embeddings:
    fEmb.write(bytes(line))

with open('overlaps.en.emb', 'wb') as eEmb:
  for line in en_embeddings:
    eEmb.write(bytes(line))

In [ ]:
## Metrics

!/content/vecalign/vecalign.py --alignment_max_size 8 --src dev.fr --tgt dev.en \
   --gold chap1.fren.txt \
   --src_embed overlaps.fr overlaps.fr.emb  \
   --tgt_embed overlaps.en overlaps.en.emb > /dev/null

 ---------------------------------
|             |  Strict |    Lax  |
| Precision   |   0.119 |   0.476 |
| Recall      |   0.156 |   0.562 |
| F1          |   0.135 |   0.516 |
 ---------------------------------


In [ ]:
!/content/vecalign/vecalign.py -h

usage: Sentence alignment using sentence embeddings and FastDTW
       [-h] -s SRC [SRC ...] -t TGT [TGT ...]
       [-g GOLD_ALIGNMENT [GOLD_ALIGNMENT ...]] --src_embed SRC_EMBED
       SRC_EMBED --tgt_embed TGT_EMBED TGT_EMBED [-a ALIGNMENT_MAX_SIZE]
       [-d DEL_PERCENTILE_FRAC] [-v] [--max_size_full_dp MAX_SIZE_FULL_DP]
       [--costs_sample_size COSTS_SAMPLE_SIZE]
       [--num_samps_for_norm NUM_SAMPS_FOR_NORM]
       [--search_buffer_size SEARCH_BUFFER_SIZE]
       [--debug_save_stack DEBUG_SAVE_STACK]

optional arguments:
  -h, --help            show this help message and exit
  -s SRC [SRC ...], --src SRC [SRC ...]
                        preprocessed source file to align (default: None)
  -t TGT [TGT ...], --tgt TGT [TGT ...]
                        preprocessed target file to align (default: None)
  -g GOLD_ALIGNMENT [GOLD_ALIGNMENT ...], --gold_alignment GOLD_ALIGNMENT [GOLD_ALIGNMENT ...]
                        preprocessed target file to align (default: None)
  --src_

In [ ]:
import sys

In [ ]:
sys.stdout = open('VecalignLog.txt','wt')

!/content/vecalign/vecalign.py --src /content/dev.fr --tgt /content/dev.en \
--src_embed /content/overlaps.fr /content/overlaps.fr.emb \
--tgt_embed /content/overlaps.en /content/overlaps.en.emb

sys.stdout.close()
sys.stdout = sys.__stdout__

In [ ]:
with open('/content/VecalignLog.txt','r') as val:
  vec_log = val.readlines()

print(type(vec_log))

In [ ]:
aligned_sent = []
for line in vec_log:
  for pair in re.findall('\[..*\]',line):
    tup_pair = []
    for item in pair.split(':'):
      if ',' in item:
        many = []
        for num in item.split(','):
          many.append(int(num.strip().replace('[','').replace(']','')))
        tup_pair.append(tuple(many))
      else:
        if item != '[]':
          tup_pair.append(int(item.replace('[','').replace(']','')))
        else:
          tup_pair.append('None')
    aligned_sent.append(tuple(tup_pair))

for tup in aligned_sent:
  if tup[0] == 'None':
    print(tup)

In [ ]:
def createAlignedUnaligned(log_file):
  aligned_sent = []

  for line in log_file:
    for pair in re.findall('\[..*\]',line):
      tup_pair = []
      for item in pair.split(':'):
        if ',' in item:
          many = []
          for num in item.split(','):
            many.append(int(num.strip().replace('[','').replace(']','')))
          tup_pair.append(tuple(many))
        else:
          if item != '[]':
            tup_pair.append(int(item.replace('[','').replace(']','')))
          else:
            tup_pair.append('None')
      aligned_sent.append(tuple(tup_pair))

  unaligned_sent_x = []
  unaligned_sent_y = []

  for tup in aligned_sent:
    if tup[1] == 'None':
      unaligned_sent_x.append(tup[0])
      #aligned_sent.remove(tup)
    elif tup[0] == 'None':
      unaligned_sent_y.append(tup[1])
      #aligned_sent.remove(tup)
  
  for tup in aligned_sent:
    if ((tup[0] == 'None') or (tup[1] == 'None')):
      aligned_sent.remove(tup)

  aligned_sent =aligned_sent[:8] + aligned_sent[9:] ## NEED BETTER METHOD WILL NOT WORK FOR LATIN

  return aligned_sent, unaligned_sent_x, unaligned_sent_y

In [ ]:
aligned_sent, fr_unaligned, eng_unaligned = createAlignedUnaligned(vec_log)

In [ ]:
aligned_sent

In [ ]:
aligned_sent_rev = [(t[1],t[0]) for t in aligned_sent]
aligned_sent_rev

In [ ]:
## FOR JS -- Sentence View
aligned_for_json = [] 
for tup in aligned_sent:
  new_tup = []
  if type(tup[0]) == tuple:
    res = ''
    for i in tup[0]:
      res = res+str(i)
    new_tup.append(res)
    if type(tup[1]) == tuple:
      ult = ''
      for i in tup[1]:
        ult = ult+str(i)
      new_tup.append(ult)
    else:
      new_tup.append(str(tup[1]))
    aligned_for_json.append(tuple(new_tup))
  else:
    new_tup.append(str(tup[0]))
    if type(tup[1]) == tuple:
      ult = ''
      for i in tup[1]:
        ult = ult+str(i)
      new_tup.append(ult)
    else:
      new_tup.append(str(tup[1]))
    aligned_for_json.append(tuple(new_tup))

with open(f'aligned_indices.json', 'w', encoding='utf-8') as json_file:
  json.dump(aligned_for_json, json_file, ensure_ascii=False, indent=4)

In [ ]:
with open('chap1.fren.txt','r',encoding='utf-8') as gold:
  vec_gold = gold.readlines()

aligned_sent, fr_unaligned, eng_unaligned = createAlignedUnaligned(vec_gold)

In [ ]:

with open("dev.fr", "r") as french_read:
  french_sent = french_read.readlines()

with open("dev.en", "r") as english_read:
  english_sent = english_read.readlines()

In [ ]:
def AwesomeAlign(file, source_text_index):
  INPUT_FILE=file
  OUTPUT_FILE=f'aligned_numbers/number{source_text_index}.src-tgt.txt'
  OUTPUT_WORD_FILE=f'aligned_words/word{source_text_index}.src-tgt.txt'
  MODEL=f'/content/drive/MyDrive/models' 

  !python /usr/local/lib/python3.7/dist-packages/awesome_align/run_align.py \
    --data_file=$INPUT_FILE \
    --output_file=$OUTPUT_FILE \
    --output_word_file=$OUTPUT_WORD_FILE \
    --model_name_or_path=$MODEL \

In [ ]:
def makeFreWordDictionary(aligned_tup, text, french_sent, english_sent, aligned_sent, unaligned_id, chap_number):
  tup_type = type(aligned_tup[0])
  if text is french_sent:
    if (type(aligned_tup[0]) == int):
      source_text_index = aligned_tup[0]
      print(str(source_text_index)+"/"+str(len(french_sent)-1))
    else:
      if len(aligned_tup[0]) == 2:
        source_text_index = str(aligned_tup[0][0]) + str(aligned_tup[0][1])
        print(str(source_text_index)+"/"+str(len(french_sent)-1))
      elif len(aligned_tup[0]) == 3:
        source_text_index = str(aligned_tup[0][0]) + str(aligned_tup[0][1]) + str(aligned_tup[0][2])
        print(str(source_text_index)+"/"+str(len(french_sent)-1))
      else:
        source_text_index = str(aligned_tup[0][0]) + str(aligned_tup[0][1]) + str(aligned_tup[0][2]) + str(aligned_tup[0][3])
        print(str(source_text_index)+"/"+str(len(french_sent)-1))
  else:
    if (type(aligned_tup[1]) == int):
      source_text_index = aligned_tup[1]
      print(str(source_text_index)+"/"+str(len(english_sent)-1))
    else:
      source_text_index = str(aligned_tup[1][0]) + str(aligned_tup[1][1])
      print(str(source_text_index)+"/"+str(len(english_sent)-1))
    
  if ((type(aligned_tup[0]) == int) and (type(aligned_tup[1]) == int)):
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(french_sent[aligned_tup[0]].replace('\n','').lstrip()+'   ||| '+english_sent[aligned_tup[1]].replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  elif ((type(aligned_tup[0]) == int) and (type(aligned_tup[1]) == tuple)):
    eng_text = ''
    for i in aligned_tup[1]:
      eng_text = eng_text + english_sent[i]
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(french_sent[aligned_tup[0]].replace('\n','').lstrip()+'   ||| '+eng_text.replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  elif ((type(aligned_tup[0]) == tuple) and (type(aligned_tup[1]) == int)):
    fr_text = ''
    for i in aligned_tup[0]:
      fr_text = fr_text + french_sent[i]
    src_tgt = open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(fr_text.replace('\n','').lstrip()+'   ||| '+english_sent[aligned_tup[1]].replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  else:
    fr_text = ''
    for i in aligned_tup[0]:
      fr_text = fr_text + french_sent[i]
    eng_text = ''
    for i in aligned_tup[1]:
      eng_text = eng_text + english_sent[i]
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(fr_text.replace('\n','').lstrip()+'   ||| '+eng_text.replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)

  align_pairs = []
  align_indices = []

  with open(f'aligned_numbers/number{source_text_index}.src-tgt.txt') as g:
    align_pairs = g.readlines()

  for pair in align_pairs:
    for p in pair.replace('  \n','').split(' '):
      tup = (int(p.split('-')[0]),int(p.split('-')[1].replace('\n', '')))
      align_indices.append(tup)
  align_indices = sorted(align_indices, key=lambda x: x[0])

  with open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt') as h:
    aligned_string = h.read()
  x_lang = [tokenizer.tokenize(word) for word in aligned_string.split('   ||| ')[0].strip().split()]
  y_lang = [tokenizer.tokenize(word) for word in aligned_string.split('   ||| ')[1].strip().split()]

  x_lang = [''.join(i).replace('##','').replace('[UNK]', '') for i in x_lang]
  y_lang = [''.join(i).replace('##','').replace('[UNK]', '') for i in y_lang]

  align_words = [] 
  for i,j in align_indices:
    align_words.append((''.join(x_lang[i]).replace('##','').replace('[UNK]', ''),''.join(y_lang[j]).replace('##','').replace('[UNK]', ''))) 

  x_lang_freq = {}
  for i, word in enumerate(x_lang):
    if word in x_lang_freq:
      x_lang_freq[word].append(i)
    else:
      x_lang_freq[word] = []
      x_lang_freq[word].append(i)

  x_lang_dict = formatJSONX(x_lang, x_lang_freq, align_words, source_text_index, aligned_sent, unaligned_id, tup_type, chap_number)

  return x_lang_dict

In [ ]:
def makeEngWordDictionary(aligned_tup, text, french_sent, english_sent, aligned_sent, unaligned_id, chap_number):
  tup_type = type(aligned_tup[1])
  if text is french_sent:
    if (type(aligned_tup[0]) == int):
      source_text_index = aligned_tup[0]
      print(str(source_text_index)+"/"+str(len(french_sent)-1))
    else:
      source_text_index = str(aligned_tup[0][0]) + str(aligned_tup[0][1])
      print(str(source_text_index)+"/"+str(len(french_sent)-1))
  else:
    if (type(aligned_tup[1]) == int):
      source_text_index = aligned_tup[1]
      print(str(source_text_index)+"/"+str(len(english_sent)-1))
    else:
      if len(aligned_tup[1]) == 2:
        source_text_index = str(aligned_tup[1][0]) + str(aligned_tup[1][1])
      elif len(aligned_tup[1]) == 3:
        source_text_index = str(aligned_tup[1][0]) + str(aligned_tup[1][1]) + str(aligned_tup[1][2])
      else:
        source_text_index = str(aligned_tup[1][0]) + str(aligned_tup[1][1]) + str(aligned_tup[1][2]) + str(aligned_tup[1][3])
      print(str(source_text_index)+"/"+str(len(english_sent)-1))
    
  if ((type(aligned_tup[0]) == int) and (type(aligned_tup[1]) == int)):
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(french_sent[aligned_tup[0]].replace('\n','').lstrip()+'   ||| '+english_sent[aligned_tup[1]].replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  elif ((type(aligned_tup[0]) == int) and (type(aligned_tup[1]) == tuple)):
    eng_text = ''
    for i in aligned_tup[1]:
      eng_text = eng_text + english_sent[i]
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(french_sent[aligned_tup[0]].replace('\n','').lstrip()+'   ||| '+eng_text.replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  elif ((type(aligned_tup[0]) == tuple) and (type(aligned_tup[1]) == int)):
    fr_text = ''
    for i in aligned_tup[0]:
      fr_text = fr_text + french_sent[i]
    src_tgt = open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(fr_text.replace('\n','').lstrip()+'   ||| '+english_sent[aligned_tup[1]].replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  else:
    fr_text = ''
    for i in aligned_tup[0]:
      fr_text = fr_text + french_sent[i]
    eng_text = ''
    for i in aligned_tup[1]:
      eng_text = eng_text + english_sent[i]
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(fr_text.replace('\n','').lstrip()+'   ||| '+eng_text.replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)

  align_pairs = []
  align_indices = []

  with open(f'aligned_numbers/number{source_text_index}.src-tgt.txt') as g:
    align_pairs = g.readlines()

  for pair in align_pairs:
    for p in pair.replace('  \n','').split(' '):
      tup = (int(p.split('-')[0]),int(p.split('-')[1].replace('\n', '')))
      align_indices.append(tup)
  align_indices = sorted(align_indices, key=lambda x: x[0])

  with open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt') as h:
    aligned_string = h.read()
  x_lang = [tokenizer.tokenize(word) for word in aligned_string.split('   ||| ')[0].strip().split()]
  y_lang = [tokenizer.tokenize(word) for word in aligned_string.split('   ||| ')[1].strip().split()]

  x_lang = [''.join(i).replace('##','').replace('[UNK]', '') for i in x_lang]
  y_lang = [''.join(i).replace('##','').replace('[UNK]', '') for i in y_lang]

  align_words = [] 
  for i,j in align_indices:
    align_words.append((''.join(x_lang[i]).replace('##','').replace('[UNK]', ''),''.join(y_lang[j]).replace('##','').replace('[UNK]', ''))) 

  y_lang_freq = {}
  for i, word in enumerate(y_lang):
    if word in y_lang_freq:
      y_lang_freq[word].append(i)
    else:
      y_lang_freq[word] = []
      y_lang_freq[word].append(i)

  y_lang_dict = formatEngJSON(y_lang, y_lang_freq, align_words, align_indices, source_text_index, aligned_sent, unaligned_id, tup_type, chap_number)

  return y_lang_dict

In [ ]:
def formatJSONX(lang, lang_freq, align_words, source_text_index, aligned_sent, unaligned_id, tup_type, chap_number):
  lang_dict = {}

  if (((len(str(source_text_index)) == 2) or (len(str(source_text_index)) == 3)) and (tup_type == tuple)):  ## two characters + len == 2 
    org_tup = tuple([int(n) for n in re.findall('.', str(source_text_index))])
    org_indx = list(zip(*aligned_sent))[0].index(org_tup)
    org_tgt = list(zip(*aligned_sent))[1][org_indx]
    sent_aligned = org_tgt 
  else:
    if len(re.findall('..', str(source_text_index))) > 1:
      org_tup = tuple([int(n) for n in re.findall('..', str(source_text_index))])
      org_indx = list(zip(*aligned_sent))[0].index(org_tup)
      org_tgt = list(zip(*aligned_sent))[1][org_indx]
      if type(org_tgt) == tuple:
        sent_aligned = ''
        for i in org_tgt:
          sent_aligned = sent_aligned + str(i)
      else:
        sent_aligned = org_tgt
    else:
      plchldr_indx = list(zip(*aligned_sent))[0].index(source_text_index)
      plchldr = list(zip(*aligned_sent))[1][plchldr_indx]
      if type(plchldr) == tuple:
        sent_aligned = ''
        for i in plchldr:
          sent_aligned = sent_aligned + str(i)
      else:
        sent_aligned = plchldr
    
  for word in lang:
    if len(lang_freq[word]) == 1: ##if unique word in the sentence
      lang_dict[f'{word}'] = [] 
      if word in list(dict(align_words).keys()):
        lang_dict[f'{word}'].append({
          'chap_number': chap_number,
          'word_text': f'{word}',
          'word_id': int(f'{lang.index(word)}'),
          'sent_id': f'{source_text_index}',
          'unaligned_id': unaligned_id,
          'en_sent_aligned': sent_aligned
          # 'align_id': f'{list(dict(align_words).keys()).index(word)}' 
        })
      else:
        lang_dict[f'{word}'].append({
          'chap_number': chap_number,
          'word_text': f'{word}',
          'word_id': int(f'{lang.index(word)}'),
          'sent_id': f'{source_text_index}',
          'unaligned_id': unaligned_id,
          'en_sent_aligned': sent_aligned 
          # 'align_id': f'None' 
        })
    else: ##if nonunique word in the sentence
      for index in lang_freq[word]:
        lang_dict[f'{word}{index}'] = []
        lang_dict[f'{word}{index}'].append({
          'chap_number': chap_number,
          'word_text': f'{word}',
          'word_id': int(f'{index}'),
          'sent_id': f'{source_text_index}',
          'unaligned_id': unaligned_id,
          'en_sent_aligned': sent_aligned
          # 'align_id': f'None' 
        })

  lang_dict = sorted(lang_dict.items(), key=lambda i: i[1][0]['word_id'])

  for sentence in nlp_fra(" ".join(lang)).sentences:
    for word in sentence.words:
      for key in lang_dict:
        if ((key[1][0]['word_text'] == word.text) or key[1][0]['word_text'][:-1] == word.text):
          key[1][0].update(word.to_dict())

  return lang_dict

In [ ]:
def formatEngJSON(lang, lang_freq, align_words, align_indices, source_text_index, aligned_sent, unaligned_id, tup_type, chap_number):
  lang_dict = {}

  if (((len(str(source_text_index)) == 2) or (len(str(source_text_index)) == 3)) and (tup_type == tuple)):  ## two characters + len == 2 
    org_tup = tuple([int(n) for n in re.findall('.', str(source_text_index))])
    org_indx = list(zip(*aligned_sent))[1].index(org_tup)
    org_tgt = list(zip(*aligned_sent))[0][org_indx]
    sent_aligned = org_tgt 
  else:
    if len(re.findall('..', str(source_text_index))) > 1:
      org_tup = tuple([int(n) for n in re.findall('..', str(source_text_index))])
      org_indx = list(zip(*aligned_sent))[1].index(org_tup)
      org_tgt = list(zip(*aligned_sent))[0][org_indx]
      if type(org_tgt) == tuple:
        sent_aligned = ''
        for i in org_tgt:
          sent_aligned = sent_aligned + str(i)
      else:
        sent_aligned = org_tgt
    else:
      plchldr_indx = list(zip(*aligned_sent))[1].index(source_text_index)
      plchldr = list(zip(*aligned_sent))[0][plchldr_indx]
      if type(plchldr) == tuple:
        sent_aligned = ''
        for i in plchldr:
          sent_aligned = sent_aligned + str(i)
      else:
        sent_aligned = plchldr

  for word in lang:
    if len(lang_freq[word]) == 1: ##if unique word in the sentence
      lang_dict[f'{word}'] = [] 
      if word in list(dict(align_words).values()):
        lang_dict[f'{word}'].append({
          'chap_number': chap_number,
          'word_text': f'{word}',
          'word_id': int(f'{lang.index(word)}'),
          'sent_id': f'{source_text_index}',
          'unaligned_id': unaligned_id,
          'sent_aligned': sent_aligned,
          'align_id': f'{list(dict(align_indices).values()).index(lang.index(word))}' 
        })
      else:
        lang_dict[f'{word}'].append({
          'chap_number': chap_number,
          'word_text': f'{word}',
          'word_id': int(f'{lang.index(word)}'),
          'sent_id': f'{source_text_index}',
          'unaligned_id': unaligned_id,
          'sent_aligned': sent_aligned, 
          'align_id': f'None' 
        })
    else: ##if nonunique word in the sentence
      for i in lang_freq[word]:
        if i in list(dict(align_indices).values()):
          tup_i = list(dict(align_indices).values()).index(i) 
          tup = align_indices[tup_i]
          lang_dict[f'{word}{i}'] = []
          lang_dict[f'{word}{i}'].append({
            'chap_number': chap_number,
            'word_text': f'{word}',
            'word_id': int(f'{i}'),
            'sent_id': f'{source_text_index}', 
            'unaligned_id': unaligned_id,
            'sent_aligned': sent_aligned,
            'align_id': f'{tup[0]}' 
          })
        else:
          lang_dict[f'{word}{i}'] = []
          lang_dict[f'{word}{i}'].append({
            'chap_number': chap_number,
            'word_text': f'{word}',
            'word_id': int(f'{i}'),
            'sent_id': f'{source_text_index}',
            'unaligned_id': unaligned_id, 
            'sent_aligned': sent_aligned,
            'align_id': f'None' 
          })

  lang_dict = sorted(lang_dict.items(), key=lambda i: i[1][0]['word_id'])

  for sentence in nlp_eng(" ".join(lang)).sentences: 
    for word in sentence.words:
      for key in lang_dict:
        if ((key[1][0]['word_text'] == word.text) or key[1][0]['word_text'][:-1] == word.text):
          key[1][0].update(word.to_dict())

  return lang_dict

In [ ]:
def formatUnalignedLineJSON(lang, lang_freq, source_text_index, lang_sent, unaligned_id, french_sent, english_sent, chap_number):
  lang_dict = {}

  if lang_sent is french_sent:
    for word in lang:
      if len(lang_freq[word]) == 1: ##if unique word in the sentence
        lang_dict[f'{word}'] = []
        lang_dict[f'{word}'].append({
          'chap_number': chap_number, 
          'word_text': f'{word}',
          'word_id': int(f'{lang.index(word)}'),
          'sent_id': int(f'{source_text_index}'),
          'unaligned_id': unaligned_id,
          'en_sent_aligned': None,
          'la_sent_aligned': None,
          'align_id': f'None' 
          })
      else: ##if nonunique word in the sentence ##MUST COME BACK!!!!!!
        for index in lang_freq[word]:
          lang_dict[f'{word}{index}'] = []
          lang_dict[f'{word}{index}'].append({
            'chap_number': chap_number,
            'word_text': f'{word}',
            'word_id': int(f'{index}'),
            'sent_id': int(f'{source_text_index}'),
            'unaligned_id': unaligned_id,
            'sent_aligned': None,
            'la_sent_aligned': None,
            'align_id': f'None' 
          })
  else:
    for word in lang:
      if len(lang_freq[word]) == 1: ##if unique word in the sentence
        lang_dict[f'{word}'] = []
        lang_dict[f'{word}'].append({
          'chap_number': chap_number,
          'word_text': f'{word}',
          'word_id': int(f'{lang.index(word)}'),
          'sent_id': int(f'{source_text_index}'),
          'unaligned_id': unaligned_id,
          'sent_aligned': None,
          'align_id': f'None' 
          })
      else: ##if nonunique word in the sentence ##MUST COME BACK!!!!!!
        for index in lang_freq[word]:
          lang_dict[f'{word}{index}'] = []
          lang_dict[f'{word}{index}'].append({
            'chap_number': chap_number,
            'word_text': f'{word}',
            'word_id': int(f'{index}'),
            'sent_id': int(f'{source_text_index}'),
            'unaligned_id': unaligned_id,
            'sent_aligned': None,
            'align_id': f'None' 
          })

  lang_dict = sorted(lang_dict.items(), key=lambda i: i[1][0]['word_id'])

  if lang_sent is french_sent:
    for sentence in nlp_fra(" ".join(lang)).sentences: ##generalize
      for word in sentence.words:
        for key in lang_dict:
          if ((key[1][0]['word_text'] == word.text) or key[1][0]['word_text'][:-1] == word.text):
            key[1][0].update(word.to_dict())
  elif lang_sent is english_sent:
    for sentence in nlp_eng(" ".join(lang)).sentences: ##generalize
      for word in sentence.words:
        for key in lang_dict:
          if ((key[1][0]['word_text'] == word.text) or key[1][0]['word_text'][:-1] == word.text):
            key[1][0].update(word.to_dict())
  else:
    for sentence in nlp_la(" ".join(lang)).sentences: ##generalize
      for word in sentence.words:
        for key in lang_dict:
          if ((key[1][0]['word_text'] == word.text) or key[1][0]['word_text'][:-1] == word.text):
            key[1][0].update(word.to_dict())

  return lang_dict

In [ ]:
def generateJSON(lang_sent, unaligned, aligned_sent, chapter_number):
  !mkdir aligned_sentences
  !mkdir aligned_numbers
  !mkdir aligned_words

  dict_to_json = {}

  for index, sentence in enumerate(lang_sent):
    dict_to_json[index] = []
    if index in unaligned:
      lang_freq = {}
      lang = lang_sent[index].split(' ')
      for i, word in enumerate(lang):
        if word in lang_freq:
          lang_freq[word].append(i)
        else:
          lang_freq[word] = []
          lang_freq[word].append(i)
      lang_dict = formatUnalignedLineJSON(lang, lang_freq, index, lang_sent, index, french_sent, english_sent, chapter_number)

      dict_to_json[index].append(lang_dict)
      print(str(index)+ "/" + str(len(lang_sent)), len(dict_to_json[index]))
    else:
      if lang_sent is french_sent:
        for tup in aligned_sent: 
          if ((type(tup[0]) == int) and tup[0] == index):
            x_lang_dict = makeFreWordDictionary(tup, french_sent, french_sent, english_sent, aligned_sent, index, chapter_number) ##THINK ABOUT BETTER PARAMETERS
            dict_to_json[index].append(x_lang_dict)
          else:
            if ((type(tup[0]) ==  tuple) and (index in tup[0])):
              x_lang_dict = makeFreWordDictionary(tup, french_sent, french_sent, english_sent, aligned_sent, index, chapter_number)
              dict_to_json[index].append(x_lang_dict)
        with open(f'fren_data_stanza{chapter_number}.json', 'w', encoding='utf-8') as lang:
          json.dump(dict_to_json, lang, ensure_ascii=False, indent=4)
      else:
        for tup in aligned_sent: 
          if ((type(tup[1]) == int) and tup[1] == index):
            y_lang_dict = makeEngWordDictionary(tup, english_sent, french_sent, english_sent, aligned_sent, index, chapter_number)
            dict_to_json[index].append(y_lang_dict)
          else:
            if ((type(tup[1]) ==  tuple) and (index in tup[1])):
              y_lang_dict = makeEngWordDictionary(tup, english_sent, french_sent, english_sent, aligned_sent, index, chapter_number)
              dict_to_json[index].append(y_lang_dict)
        with open(f'en_data_stanza{chapter_number}.json', 'w', encoding='utf-8') as lang:
          json.dump(dict_to_json, lang, ensure_ascii=False, indent=4)

  !rm -r /content/aligned_numbers*
  !rm -r /content/aligned_sentences*
  !rm -r /content/aligned_words*

  return dict_to_json

In [ ]:
generateJSON(french_sent, fr_unaligned, aligned_sent, 0)

mkdir: cannot create directory ‘aligned_sentences’: File exists
mkdir: cannot create directory ‘aligned_numbers’: File exists
mkdir: cannot create directory ‘aligned_words’: File exists
0/44
Loading the dataset...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Extracting: 1it [00:00,  1.79it/s]
1/44
Loading the dataset...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. 

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


3/44
Loading the dataset...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Extracting: 1it [00:00,  1.00it/s]
4/44
Loading the dataset...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked

{0: [[('REPUBLIQUE',
    [{'chap_number': 0,
      'deprel': 'nsubj',
      'en_sent_aligned': 0,
      'end_char': 10,
      'head': 4,
      'id': 1,
      'lemma': 'REPUBLIQUE',
      'sent_id': '0',
      'start_char': 0,
      'text': 'REPUBLIQUE',
      'unaligned_id': 0,
      'upos': 'PROPN',
      'word_id': 0,
      'word_text': 'REPUBLIQUE'}]),
   ('est1',
    [{'chap_number': 0,
      'deprel': 'cop',
      'en_sent_aligned': 0,
      'end_char': 80,
      'feats': 'Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
      'head': 16,
      'id': 15,
      'lemma': 'être',
      'sent_id': '0',
      'start_char': 77,
      'text': 'est',
      'unaligned_id': 0,
      'upos': 'AUX',
      'word_id': 1,
      'word_text': 'est'}]),
   ('un',
    [{'chap_number': 0,
      'deprel': 'det',
      'en_sent_aligned': 0,
      'end_char': 17,
      'feats': 'Definite=Ind|Gender=Masc|Number=Sing|PronType=Art',
      'head': 4,
      'id': 3,
      'lemma': 'un',
      'sent_id

In [ ]:
generateJSON(english_sent, eng_unaligned, aligned_sent, 0)

mkdir: cannot create directory ‘aligned_sentences’: File exists
mkdir: cannot create directory ‘aligned_numbers’: File exists
mkdir: cannot create directory ‘aligned_words’: File exists
0/65
Loading the dataset...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Extracting: 1it [00:00,  1.65it/s]
1/65
Loading the dataset...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. 

IndexError: ignored

# Latin

In [ ]:
import re
doc = docx.Document('full book of latin.docx')

bodin_la = []

for p in doc.paragraphs:
  if (('Page' not in p.text) and ('PAGE' not in p.text) and (not p.text.isnumeric())):
    bodin_la.append(p.text)
   
bodin_la_string = ('\n'.join(bodin_la)).replace("\n","").replace("\t","")
x = re.split("<HI>[\s\S]*?<\/HI>",bodin_la_string)

# y = re.search("<DIV1.+",x[1].split("</HEAD>")[1])
# ch_1_text = x[1].split("</HEAD>")[1][0:y.start()]

# text_clean = re.sub("\[(.*?)\]", ' ', ch_1_text).replace("  ", "").replace("- -", " ")
# lat_soup = BeautifulSoup(text_clean, 'lxml')
# for note in lat_soup.find_all('note'):
#   note.decompose()
# lat_soup.get_text()
lat_dict = {}
for index, caput in enumerate(x[1:]):

  title = caput.split("</HEAD>")[0].replace("?","").strip()

  y = re.search("<DIV1.+",caput.split("</HEAD>")[1])
  if y:
    ch_text = caput.split("</HEAD>")[1][0:y.start()]
    text_clean = re.sub("\[(.*?)\]", ' ', ch_text).replace(",",",  ").replace("  ", "").replace("- -", " ").replace(":", ": ")
    lat_soup = BeautifulSoup(text_clean, 'lxml')
    for note in lat_soup.find_all('note'):
      note.decompose()
    text_final = lat_soup.get_text()
  else:
    ch_text = caput.split("</HEAD>")[1]
    text_clean = re.sub("\[(.*?)\]", ' ', ch_text).replace(",",", ").replace("  ", "").replace("- -", " ").replace(":", ": ")
    lat_soup = BeautifulSoup(text_clean, 'lxml')
    for note in lat_soup.find_all('note'):
      note.decompose()
    text_final = lat_soup.get_text()
  lat_dict[f'Chapter {index+1}'] = text_final.strip()

latin_df = pd.DataFrame.from_dict(lat_dict, orient='index')
split_lines = re.findall("[A-Z].+?\.", latin_df[0][0])#############COMEBACK  [f(x) for x in sequence if condition]
split_lines = [' '.join([split_lines[i], split_lines[i+1]]) if len(split_lines[i].split())<4 else split_lines[i] for i in range(len(split_lines))] 

with open('dev.la', 'w') as fDev:
    for line in split_lines:
      fDev.write(line + '\n')

In [ ]:
!git clone https://github.com/thompsonb/vecalign

!/content/vecalign/overlap.py -i /content/dev.fr -o overlaps.fr -n 10
!/content/vecalign/overlap.py -i /content/dev.la -o overlaps.la -n 10

!python -m laserembeddings download-models
from laserembeddings import Laser

laser = Laser()

with open('/content/overlaps.fr', 'r') as frOver:
  fr_overlap = frOver.readlines() 

with open('/content/overlaps.la', 'r') as laOver:
  la_overlap = laOver.readlines()

fr_embeddings = laser.embed_sentences(fr_overlap,lang='fr')
la_embeddings = laser.embed_sentences(la_overlap,lang='la')

with open('overlaps.fr.emb','wb') as fEmb:
  for line in fr_embeddings:
    fEmb.write(bytes(line))

In [ ]:
with open('overlaps.la.emb', 'wb') as lEmb:
  for line in la_embeddings:
    lEmb.write(bytes(line))

import sys
sys.stdout = open('VecalignLog.txt','wt')

!/content/vecalign/vecalign.py --src /content/dev.fr --tgt /content/dev.la \
--src_embed /content/overlaps.fr /content/overlaps.fr.emb \
--tgt_embed /content/overlaps.la /content/overlaps.la.emb

sys.stdout.close()
sys.stdout = sys.__stdout__

In [ ]:
def createAlignedUnaligned(log_file):
  aligned_sent = []

  for line in log_file:
    for pair in re.findall('\[..*\]',line):
      tup_pair = []
      for item in pair.split(':'):
        if ',' in item:
          many = []
          for num in item.split(','):
            many.append(int(num.strip().replace('[','').replace(']','')))
          tup_pair.append(tuple(many))
        else:
          if item != '[]':
            tup_pair.append(int(item.replace('[','').replace(']','')))
          else:
            tup_pair.append('None')
      aligned_sent.append(tuple(tup_pair))

  unaligned_sent_x = []
  unaligned_sent_y = []

  for tup in aligned_sent:
    if tup[1] == 'None':
      unaligned_sent_x.append(tup[0])
      #aligned_sent.remove(tup)
    elif tup[0] == 'None':
      unaligned_sent_y.append(tup[1])
      #aligned_sent.remove(tup)
  
  for tup in aligned_sent:
    if ((tup[0] == 'None') or (tup[1] == 'None')):
      aligned_sent.remove(tup)

  #aligned_sent =aligned_sent[:8] + aligned_sent[9:] ## NEED BETTER METHOD WILL NOT WORK FOR LATIN

  return aligned_sent, unaligned_sent_x, unaligned_sent_y

In [ ]:
with open('/content/VecalignLog.txt','r') as val:
  vec_log = val.readlines()

aligned_sent, fr_unaligned, la_unaligned = createAlignedUnaligned(vec_log)

In [ ]:
fr_unaligned

[32]

In [ ]:
la_unaligned

[63]

In [ ]:
aligned_sent

[(0, 0),
 (1, 1),
 (2, 2),
 (3, (3, 4)),
 (4, 5),
 (5, 6),
 (6, (7, 8, 9)),
 (7, 10),
 (8, 11),
 (9, 12),
 (10, 13),
 (11, 14),
 (12, 15),
 (13, 16),
 (14, 17),
 (15, (18, 19, 20)),
 (16, 21),
 (17, (22, 23, 24)),
 (18, 25),
 (19, 26),
 (20, 27),
 (21, 28),
 (22, 29),
 (23, 30),
 (24, (31, 32, 33)),
 (25, (34, 35, 36)),
 (26, 37),
 (27, 38),
 (28, 39),
 (29, (40, 41, 42)),
 (30, 43),
 (31, (44, 45, 46)),
 (33, (47, 48)),
 (34, 49),
 (35, 50),
 (36, 51),
 (37, (52, 53, 54)),
 (38, 55),
 (39, 56),
 (40, (57, 58)),
 (41, (59, 60)),
 (42, 61),
 (43, 62),
 (44, 64)]

In [ ]:
## FOR JS -- Sentence View
aligned_for_json = [] 
for tup in aligned_sent:
  new_tup = []
  if type(tup[0]) == tuple:
    res = ''
    for i in tup[0]:
      res = res+str(i)
    new_tup.append(res)
    if type(tup[1]) == tuple:
      ult = ''
      for i in tup[1]:
        ult = ult+str(i)
      new_tup.append(ult)
    else:
      new_tup.append(str(tup[1]))
    aligned_for_json.append(tuple(new_tup))
  else:
    new_tup.append(str(tup[0]))
    if type(tup[1]) == tuple:
      ult = ''
      for i in tup[1]:
        ult = ult+str(i)
      new_tup.append(ult)
    else:
      new_tup.append(str(tup[1]))
    aligned_for_json.append(tuple(new_tup))

with open(f'la_aligned_indices.json', 'w', encoding='utf-8') as json_file:
  json.dump(aligned_for_json, json_file, ensure_ascii=False, indent=4)

In [ ]:
with open("dev.fr", "r") as french_read:
  french_sent = french_read.readlines()

with open("dev.la", "r") as latin_read:
  latin_sent = latin_read.readlines()

In [ ]:
def makeFreWordDictionary(aligned_tup, text, french_sent, english_sent, aligned_sent):
  if text is french_sent:
    if (type(aligned_tup[0]) == int):
      source_text_index = aligned_tup[0]
      print(str(source_text_index)+"/"+str(len(french_sent)-1))
    else:
      if len(aligned_tup[0]) == 2:
        source_text_index = str(aligned_tup[0][0]) + str(aligned_tup[0][1])
        print(str(source_text_index)+"/"+str(len(french_sent)-1))
      else:
        source_text_index = str(aligned_tup[0][0]) + str(aligned_tup[0][1]) + str(aligned_tup[0][2])
        print(str(source_text_index)+"/"+str(len(french_sent)-1))
  else:
    if (type(aligned_tup[1]) == int):
      source_text_index = aligned_tup[1]
      print(str(source_text_index)+"/"+str(len(english_sent)-1))
    else:
      source_text_index = str(aligned_tup[1][0]) + str(aligned_tup[1][1])
      print(str(source_text_index)+"/"+str(len(english_sent)-1))
    
  if ((type(aligned_tup[0]) == int) and (type(aligned_tup[1]) == int)):
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(french_sent[aligned_tup[0]].replace('\n','').lstrip()+'   ||| '+english_sent[aligned_tup[1]].replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  elif ((type(aligned_tup[0]) == int) and (type(aligned_tup[1]) == tuple)):
    eng_text = ''
    for i in aligned_tup[1]:
      eng_text = eng_text + english_sent[i]
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(french_sent[aligned_tup[0]].replace('\n','').lstrip()+'   ||| '+eng_text.replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  elif ((type(aligned_tup[0]) == tuple) and (type(aligned_tup[1]) == int)):
    fr_text = ''
    for i in aligned_tup[0]:
      fr_text = fr_text + french_sent[i]
    src_tgt = open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(fr_text.replace('\n','').lstrip()+'   ||| '+english_sent[aligned_tup[1]].replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  else:
    fr_text = ''
    for i in aligned_tup[0]:
      fr_text = fr_text + french_sent[i]
    eng_text = ''
    for i in aligned_tup[1]:
      eng_text = eng_text + english_sent[i]
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(fr_text.replace('\n','').lstrip()+'   ||| '+eng_text.replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)

  align_pairs = []
  align_indices = []

  with open(f'aligned_numbers/number{source_text_index}.src-tgt.txt') as g:
    align_pairs = g.readlines()

  for pair in align_pairs:
    for p in pair.replace('  \n','').split(' '):
      tup = (int(p.split('-')[0]),int(p.split('-')[1].replace('\n', '')))
      align_indices.append(tup)
  align_indices = sorted(align_indices, key=lambda x: x[0])

  with open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt') as h:
    aligned_string = h.read()
  x_lang = [tokenizer.tokenize(word) for word in aligned_string.split('   ||| ')[0].strip().split()]
  y_lang = [tokenizer.tokenize(word) for word in aligned_string.split('   ||| ')[1].strip().split()]

  x_lang = [''.join(i).replace('##','').replace('[UNK]', '') for i in x_lang]
  y_lang = [''.join(i).replace('##','').replace('[UNK]', '') for i in y_lang]

  align_words = [] 
  for i,j in align_indices:
    align_words.append((''.join(x_lang[i]).replace('##','').replace('[UNK]', ''),''.join(y_lang[j]).replace('##','').replace('[UNK]', ''))) 

  x_lang_freq = {}
  for i, word in enumerate(x_lang):
    if word in x_lang_freq:
      x_lang_freq[word].append(i)
    else:
      x_lang_freq[word] = []
      x_lang_freq[word].append(i)

  x_lang_dict = formatJSONX(x_lang, x_lang_freq, align_words, source_text_index, aligned_sent)

  return x_lang_dict

In [ ]:
def formatJSONX(lang, lang_freq, align_words, source_text_index, aligned_sent):
  lang_dict = {}

  if len(re.findall('..',str(source_text_index))) > 1:
    org_tup = tuple([int(n) for n in re.findall('..',source_text_index)])
    org_indx = list(zip(*aligned_sent))[0].index(org_tup)
    org_tgt = list(zip(*aligned_sent))[1][org_indx]
    if type(org_tgt) == tuple:
      sent_aligned = str(org_tgt[0]) + str(org_tgt[1])
    else:
      sent_aligned = org_tgt
  else:
    plchldr_indx = list(zip(*aligned_sent))[0].index(source_text_index)
    plchldr = list(zip(*aligned_sent))[1][plchldr_indx]
    if type(plchldr) == tuple:
      sent_aligned = str(plchldr[0]) + str(plchldr[1]) 
    else:
      sent_aligned = plchldr
    

  for word in lang:
    if len(lang_freq[word]) == 1: ##if unique word in the sentence
      lang_dict[f'{word}'] = [] 
      if word in list(dict(align_words).keys()):
        lang_dict[f'{word}'].append({
          'word_text': f'{word}',
          'word_id': int(f'{lang.index(word)}'),
          'sent_id': f'{source_text_index}',
          'sent_aligned': sent_aligned
          # 'align_id': f'{list(dict(align_words).keys()).index(word)}' 
        })
      else:
        lang_dict[f'{word}'].append({
          'word_text': f'{word}',
          'word_id': int(f'{lang.index(word)}'),
          'sent_id': f'{source_text_index}',
          'sent_aligned': sent_aligned 
          # 'align_id': f'None' 
        })
    else: ##if nonunique word in the sentence
      for index in lang_freq[word]:
        lang_dict[f'{word}{index}'] = []
        lang_dict[f'{word}{index}'].append({
          'word_text': f'{word}',
          'word_id': int(f'{index}'),
          'sent_id': f'{source_text_index}', 
          'sent_aligned': sent_aligned
          # 'align_id': f'None' 
        })

  lang_dict = sorted(lang_dict.items(), key=lambda i: i[1][0]['word_id'])

  for sentence in nlp_fra(" ".join(lang)).sentences:
    for word in sentence.words:
      for key in lang_dict:
        if ((key[1][0]['word_text'] == word.text) or key[1][0]['word_text'][:-1] == word.text):
          key[1][0].update(word.to_dict())

  return lang_dict

In [ ]:
def makeLatWordDictionary(aligned_tup, text, french_sent, english_sent, aligned_sent):
  if text is french_sent:
    if (type(aligned_tup[0]) == int):
      source_text_index = aligned_tup[0]
      print(str(source_text_index)+"/"+str(len(french_sent)-1))
    else:
      source_text_index = str(aligned_tup[0][0]) + str(aligned_tup[0][1])
      print(str(source_text_index)+"/"+str(len(french_sent)-1))
  else:
    if (type(aligned_tup[1]) == int):
      source_text_index = aligned_tup[1]
      print(str(source_text_index)+"/"+str(len(english_sent)-1))
    else:
      if len(aligned_tup[1]) == 2:
        source_text_index = str(aligned_tup[1][0]) + str(aligned_tup[1][1])
      else:
        source_text_index = str(aligned_tup[1][0]) + str(aligned_tup[1][1]) + str(aligned_tup[1][2])
      print(str(source_text_index)+"/"+str(len(english_sent)-1))
    
  if ((type(aligned_tup[0]) == int) and (type(aligned_tup[1]) == int)):
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(french_sent[aligned_tup[0]].replace('\n','').lstrip()+'   ||| '+english_sent[aligned_tup[1]].replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  elif ((type(aligned_tup[0]) == int) and (type(aligned_tup[1]) == tuple)):
    eng_text = ''
    for i in aligned_tup[1]:
      eng_text = eng_text + english_sent[i]
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(french_sent[aligned_tup[0]].replace('\n','').lstrip()+'   ||| '+eng_text.replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  elif ((type(aligned_tup[0]) == tuple) and (type(aligned_tup[1]) == int)):
    fr_text = ''
    for i in aligned_tup[0]:
      fr_text = fr_text + french_sent[i]
    src_tgt = open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(fr_text.replace('\n','').lstrip()+'   ||| '+english_sent[aligned_tup[1]].replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)
  else:
    fr_text = ''
    for i in aligned_tup[0]:
      fr_text = fr_text + french_sent[i]
    eng_text = ''
    for i in aligned_tup[1]:
      eng_text = eng_text + english_sent[i]
    open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt','w').write(fr_text.replace('\n','').lstrip()+'   ||| '+eng_text.replace('\n','').lstrip())
    AwesomeAlign(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt', source_text_index)

  align_pairs = []
  align_indices = []

  with open(f'aligned_numbers/number{source_text_index}.src-tgt.txt') as g:
    align_pairs = g.readlines()

  for pair in align_pairs:
    for p in pair.replace('  \n','').split(' '):
      tup = (int(p.split('-')[0]),int(p.split('-')[1].replace('\n', '')))
      align_indices.append(tup)
  align_indices = sorted(align_indices, key=lambda x: x[0])

  with open(f'aligned_sentences/sentence{source_text_index}.src-tgt.txt') as h:
    aligned_string = h.read()
  x_lang = [tokenizer.tokenize(word) for word in aligned_string.split('   ||| ')[0].strip().split()]
  y_lang = [tokenizer.tokenize(word) for word in aligned_string.split('   ||| ')[1].strip().split()]

  x_lang = [''.join(i).replace('##','').replace('[UNK]', '') for i in x_lang]
  y_lang = [''.join(i).replace('##','').replace('[UNK]', '') for i in y_lang]

  align_words = [] 
  for i,j in align_indices:
    align_words.append((''.join(x_lang[i]).replace('##','').replace('[UNK]', ''),''.join(y_lang[j]).replace('##','').replace('[UNK]', ''))) 

  y_lang_freq = {}
  for i, word in enumerate(y_lang):
    if word in y_lang_freq:
      y_lang_freq[word].append(i)
    else:
      y_lang_freq[word] = []
      y_lang_freq[word].append(i)

  y_lang_dict = formatJSONY(y_lang, y_lang_freq, align_words, source_text_index, aligned_sent)

  return y_lang_dict

In [ ]:
def formatJSONY(lang, lang_freq, align_words, source_text_index, aligned_sent):
  lang_dict = {}

  if ((source_text_index != '56') and (source_text_index != '78')): 
    if len(re.findall('..',str(source_text_index))) > 1:
      org_tup = tuple([int(n) for n in re.findall('..',source_text_index)])
      org_indx = list(zip(*aligned_sent))[1].index(org_tup)
      org_src = list(zip(*aligned_sent))[0][org_indx]
      if type(org_src) == tuple:
        sent_aligned = str(org_src[0]) + str(org_src[1])
      else:
        sent_aligned = org_src
    else:
      plchldr_indx = list(zip(*aligned_sent))[1].index(source_text_index)
      plchldr = list(zip(*aligned_sent))[0][plchldr_indx]
      if type(plchldr) == tuple:
        sent_aligned = str(plchldr[0]) + str(plchldr[1]) 
      else:
        sent_aligned = plchldr
  else:
    org_tup = tuple([int(n) for n in re.findall('.',source_text_index)])
    org_indx = list(zip(*aligned_sent))[1].index(org_tup)
    org_tgt = list(zip(*aligned_sent))[0][org_indx]
    sent_aligned = org_tgt

  for word in lang:
    if len(lang_freq[word]) == 1: ##if unique word in the sentence
      lang_dict[f'{word}'] = [] 
      if word in list(dict(align_words).values()):
        lang_dict[f'{word}'].append({
          'word_text': f'{word}',
          'word_id': int(f'{lang.index(word)}'),
          'sent_id': int(f'{source_text_index}'),
          'sent_aligned': sent_aligned,
          'align_id': f'{list(dict(align_words).values()).index(word)}' 
        })
      else:
        lang_dict[f'{word}'].append({
          'word_text': f'{word}',
          'word_id': int(f'{lang.index(word)}'),
          'sent_id': int(f'{source_text_index}'),
          'sent_aligned': sent_aligned,
          'align_id': f'None' 
        })
    else: ##if nonunique word in the sentence ##MUST COME BACK!!!!!!
      for index in lang_freq[word]:
        lang_dict[f'{word}{index}'] = []
        lang_dict[f'{word}{index}'].append({
          'word_text': f'{word}',
          'word_id': int(f'{index}'),
          'sent_id': int(f'{source_text_index}'),
          'sent_aligned': sent_aligned,
          'align_id': f'None' 
        })

  lang_dict = sorted(lang_dict.items(), key=lambda i: i[1][0]['word_id'])

  for sentence in nlp_la(" ".join(lang)).sentences: ## FIX
    for word in sentence.words:
      for key in lang_dict:
        if ((key[1][0]['word_text'] == word.text) or key[1][0]['word_text'][:-1] == word.text):
          key[1][0].update(word.to_dict())

  return lang_dict

In [ ]:
def formatUnalignedLineJSON(lang, lang_freq, source_text_index, lang_sent):
  lang_dict = {}

  for word in lang:
    if len(lang_freq[word]) == 1: ##if unique word in the sentence
      lang_dict[f'{word}'] = []
      lang_dict[f'{word}'].append({
        'word_text': f'{word}',
        'word_id': int(f'{lang.index(word)}'),
        'sent_id': int(f'{source_text_index}'),
        'sent_aligned': None,
        'align_id': f'None' 
        })
    else: ##if nonunique word in the sentence ##MUST COME BACK!!!!!!
      for index in lang_freq[word]:
        lang_dict[f'{word}{index}'] = []
        lang_dict[f'{word}{index}'].append({
          'word_text': f'{word}',
          'word_id': int(f'{index}'),
          'sent_id': int(f'{source_text_index}'),
          'sent_aligned': None,
          'align_id': f'None' 
        })

  lang_dict = sorted(lang_dict.items(), key=lambda i: i[1][0]['word_id'])

  if lang_sent is french_sent:
    for sentence in nlp_fra(" ".join(lang)).sentences: ##generalize
      for word in sentence.words:
        for key in lang_dict:
          if ((key[1][0]['word_text'] == word.text) or key[1][0]['word_text'][:-1] == word.text):
            key[1][0].update(word.to_dict())
  elif lang_sent is english_sent:
    for sentence in nlp_eng(" ".join(lang)).sentences: ##generalize
      for word in sentence.words:
        for key in lang_dict:
          if ((key[1][0]['word_text'] == word.text) or key[1][0]['word_text'][:-1] == word.text):
            key[1][0].update(word.to_dict())
  else:
    for sentence in nlp_la(" ".join(lang)).sentences: ##generalize
      for word in sentence.words:
        for key in lang_dict:
          if ((key[1][0]['word_text'] == word.text) or key[1][0]['word_text'][:-1] == word.text):
            key[1][0].update(word.to_dict())

  return lang_dict

In [ ]:
def generateJSON(lang_sent, unaligned, aligned_sent):
  !mkdir aligned_sentences
  !mkdir aligned_numbers
  !mkdir aligned_words

  dict_to_json = {}

  for index, sentence in enumerate(lang_sent):
    dict_to_json[index] = []
    if index in unaligned:
      lang_freq = {}
      lang = lang_sent[index].split(' ')
      for i, word in enumerate(lang):
        if word in lang_freq:
          lang_freq[word].append(i)
        else:
          lang_freq[word] = []
          lang_freq[word].append(i)
      lang_dict = formatUnalignedLineJSON(lang, lang_freq, index, lang_sent)

      dict_to_json[index].append(lang_dict)
      print(str(index)+ "/" + str(len(lang_sent)-1), len(dict_to_json[index]))
    else:
      if lang_sent is french_sent:
        pass
        # for tup in aligned_sent: 
        #   if ((type(tup[0]) == int) and tup[0] == index):
        #     x_lang_dict, y_lang_dict = makeWordDictionary(tup, french_sent, french_sent, latin_sent, aligned_sent) ##THINK ABOUT BETTER PARAMETERS
        #     dict_to_json[index].append(x_lang_dict)
        #   else:
        #     if ((type(tup[0]) ==  tuple) and (index in tup[0])):
        #       x_lang_dict, y_lang_dict = makeWordDictionary(tup, french_sent, french_sent, latin_sent, aligned_sent)
        #       dict_to_json[index].append(x_lang_dict)
        # with open(f'fr_data_stanza.json', 'w', encoding='utf-8') as lang:
        #   json.dump(dict_to_json, lang, ensure_ascii=False, indent=4)
      else:
        for tup in aligned_sent: 
          if ((type(tup[1]) == int) and tup[1] == index):
            y_lang_dict = makeLatWordDictionary(tup, latin_sent, french_sent, latin_sent, aligned_sent)
            dict_to_json[index].append(y_lang_dict)
          else:
            if ((type(tup[1]) ==  tuple) and (index in tup[1])):
              y_lang_dict = makeLatWordDictionary(tup, latin_sent, french_sent, latin_sent, aligned_sent)
              dict_to_json[index].append(y_lang_dict)
        with open(f'lat_data_stanza.json', 'w', encoding='utf-8') as lang:
          json.dump(dict_to_json, lang, ensure_ascii=False, indent=4)

  !rm -r /content/aligned_numbers*
  !rm -r /content/aligned_sentences*
  !rm -r /content/aligned_words*

  return dict_to_json

# New Section

In [ ]:
def readJSON(file):
  json_read = open(file)
  data = json.load(json_read)
  return data

In [ ]:
def countNone(file):
  align_ids = []

  data = readJSON(file)
  for i in data:
    for j in data[i]:
      for k in j:
        align_ids.append(k[1][0]['align_id'])

  return align_ids.count('None')/len(align_ids)

In [ ]:
countNone('/content/lat_data_stanza0.json')

0.7699530516431925